# 알츠하이머 예측하기 
## 데이터 출처
- 알츠하이머 데이터
- https://www.kaggle.com/datasets/rabieelkharoua/alzheimers-disease-dataset

# 데이터 확인
## 1. 인적 정보
|column 명|설명|value|column 명|설명|value|
|--|--|--|--|--|--|
|PatientID|식별 번호|471-6900|Age|나이|60-90|
|Gender|성별|0:남자, 1 : 여자|Ethnicity|인종|0:백인, 1 : 아프리카계 미국인, 2 : 아시아, 3 : 기타|
|EducationLevel|교육 수준|0 : 없음, 1 : 고등학교, 2 : 학사, 3 : 더 높음|

## 2. lifestype
|column 명|설명|value|column 명|설명|value|
|--|--|--|--|--|--|
|Smoking|흡연 유/무|0 : 아니오, 1 : 예|AlcoholConsumption|주당 알코올 소비량|0-20|
|PhysicalActivity|주간 신체 활동량|0-10|DietQuality|식단 품질|0-10|
|SleepQuality|수면의 질|4-10|

## 3. 병력
|column 명|설명|value|column 명|설명|value|
|--|--|--|--|--|--|
|FamilyHistoryAlzheimers|알츠하이머 가족력|0 : 없음, 1 : 예|CardiovascularDisease|심혈관 질환 유무|0 : 없음, 1 : 예|
|Diabetes|당뇨병 유무|0 : 없음, 1 : 예|Depression|우울증 유무|0 : 없음, 1 : 예|
|HeadInjury|두부 손상 유무|0 : 없음, 1 : 예|Hypertension|고혈압 유무|0 : 없음, 1 : 예|

## 4. 임상 측정
|column 명|설명|value|column 명|설명|value|
|--|--|--|--|--|--|
|SystolicBP|수축기 혈압|90-180|DiastolicBP|이완기 혈압|60-120|
|CholesterolTotal|콜레스테롤 총량|150-300|CholesterolLDL|LDL콜레스테롤 수치|50-200|
|CholesterolHDL|HDL콜레스테롤 수치|20-100|CholesterolTriglycerides|중성지방|50-400|
|BMI|신체질량지수|15-40|

## 5. 인지 및 기능 평가
|column 명|설명|value|column 명|설명|value|
|--|--|--|--|--|--|
|MMSE|간이 정신 상태 검사 점수|0-30, 낮을수록 인지 장애가 있음|FunctionalAssessment|기능평가|0-10, 낮을수록 기능 장애가 심함|
|MemoryComplaints|기억 장애 유무|0 : 없음, 1 : 예|BehavioralProblems|행동 문제 유무|0 : 없음, 1 : 예|
|ADL|일상생활활동 점수|0-10, 낮을수록 장애가 심함|

## 6. 증상
|column 명|설명|value|column 명|설명|value|
|--|--|--|--|--|--|
|Confusion|혼란의 유무|0 : 없음, 1 : 예|Disorientation|방향 감각 상실 유무|0 : 없음, 1 : 예|
|PersonalityChanges|성격 변화 유무|0 : 없음, 1 : 예|DifficultyCompletingTasks|작업 완료 어려움 유무|0 : 없음, 1 : 예|
|Forgetfulness|건망증 유무|0 : 없음, 1 : 예|

## 7. 진단
|column 명|설명|value|column 명|설명|value|
|--|--|--|--|--|--|
|Diagnosis|알츠하이머 유무|0 : 없음, 1 : 예|DoctorInCharge|담당 의사에 대한 정보|'XXXConfid'|

In [1]:
pip install statsmodels


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [108]:
import pandas as pd
import numpy as np
from scipy import stats 
from statsmodels.formula.api import logit 
from sklearn.metrics import accuracy_score,classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


df = pd.read_csv('../data/alzheimers_disease_data.csv')
df

,PatientID,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,...,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness,Diagnosis,DoctorInCharge
0,4751,73,0,0,2,22.927749,0,13.297218,6.327112,1.347214,...,0,0,1.725883,0,0,0,1,0,0,XXXConfid
1,4752,89,0,0,0,26.827681,0,4.542524,7.619885,0.518767,...,0,0,2.592424,0,0,0,0,1,0,XXXConfid
2,4753,73,0,3,1,17.795882,0,19.555085,7.844988,1.826335,...,0,0,7.119548,0,1,0,1,0,0,XXXConfid
3,4754,74,1,0,1,33.800817,1,12.209266,8.428001,7.435604,...,0,1,6.481226,0,0,0,0,0,0,XXXConfid
4,4755,89,0,0,0,20.716974,0,18.454356,6.310461,0.795498,...,0,0,0.014691,0,0,1,1,0,0,XXXConfid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2144,6895,61,0,0,1,39.121757,0,1.561126,4.049964,6.555306,...,0,0,4.492838,1,0,0,0,0,1,XXXConfid
2145,6896,75,0,0,2,17.857903,0,18.767261,1.360667,2.904662,...,0,1,9.204952,0,0,0,0,0,1,XXXConfid
2146,6897,77,0,0,1,15.476479,0,4.594670,9.886002,8.120025,...,0,0,5.036334,0,0,0,0,0,1,XXXConfid
2147,6898,78,1,3,1,15.299911,0,8.674505,6.354282,1.263427,...,0,0,3.785399,0,0,0,0,1,1,XXXConfid


In [109]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
PatientID,2149.0,5825.000000,620.507185,4751.000000,5288.000000,5825.000000,6362.000000,6899.000000
Age,2149.0,74.908795,8.990221,60.000000,67.000000,75.000000,83.000000,90.000000
Gender,2149.0,0.506282,0.500077,0.000000,0.000000,1.000000,1.000000,1.000000
Ethnicity,2149.0,0.697534,0.996128,0.000000,0.000000,0.000000,1.000000,3.000000
EducationLevel,2149.0,1.286645,0.904527,0.000000,1.000000,1.000000,2.000000,3.000000
BMI,2149.0,27.655697,7.217438,15.008851,21.611408,27.823924,33.869778,39.992767
Smoking,2149.0,0.288506,0.453173,0.000000,0.000000,0.000000,1.000000,1.000000
AlcoholConsumption,2149.0,10.039442,5.757910,0.002003,5.139810,9.934412,15.157931,19.989293
PhysicalActivity,2149.0,4.920202,2.857191,0.003616,2.570626,4.766424,7.427899,9.987429
DietQuality,2149.0,4.993138,2.909055,0.009385,2.458455,5.076087,7.558625,9.998346


# 데이터 전처리

## 결측치 확인 : 없음

In [110]:
df.isnull().sum()

PatientID                    0
Age                          0
Gender                       0
Ethnicity                    0
EducationLevel               0
BMI                          0
Smoking                      0
AlcoholConsumption           0
PhysicalActivity             0
DietQuality                  0
SleepQuality                 0
FamilyHistoryAlzheimers      0
CardiovascularDisease        0
Diabetes                     0
Depression                   0
HeadInjury                   0
Hypertension                 0
SystolicBP                   0
DiastolicBP                  0
CholesterolTotal             0
CholesterolLDL               0
CholesterolHDL               0
CholesterolTriglycerides     0
MMSE                         0
FunctionalAssessment         0
MemoryComplaints             0
BehavioralProblems           0
ADL                          0
Confusion                    0
Disorientation               0
PersonalityChanges           0
DifficultyCompletingTasks    0
Forgetfu

## 필요없는 column 제거 : PatientID, DoctorInCharge

In [111]:
df.drop(['PatientID','DoctorInCharge'],axis=1, inplace=True)

## 데이터 정규화

In [112]:
scaler = MinMaxScaler()
for i in list['Age','BMI','AlcoholConsumption','PhysicalActivity','DietQuality','SleepQuality','SystolicBP','DiastolicBP','DiastolicBP','CholesterolTotal','CholesterolLDL','CholesterolHDL','CholesterolTriglycerides','MMSE','FunctionalAssessment','ADL']:
    df[i] = scaler.fit_transform(df[i])
df.describe()

,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,SleepQuality,...,FunctionalAssessment,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness,Diagnosis
count,2149.000000,2149.000000,2149.000000,2149.000000,2149.000000,2149.000000,2149.000000,2149.000000,2149.000000,2149.000000,...,2149.000000,2149.000000,2149.000000,2149.000000,2149.000000,2149.000000,2149.000000,2149.000000,2149.000000,2149.000000
mean,0.496960,0.506282,0.232511,0.428882,0.506199,0.288506,0.502191,0.492456,0.498926,0.508312,...,0.508162,0.208004,0.156817,0.498244,0.205212,0.158213,0.150768,0.158678,0.301536,0.353653
std,0.299674,0.500077,0.332043,0.301509,0.288883,0.453173,0.288079,0.286182,0.291227,0.294065,...,0.289390,0.405974,0.363713,0.295023,0.403950,0.365026,0.357906,0.365461,0.459032,0.478214
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.233333,0.000000,0.000000,0.333333,0.264272,0.000000,0.257054,0.257117,0.245178,0.246843,...,0.256685,0.000000,0.000000,0.234191,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.500000,1.000000,0.000000,0.333333,0.512933,0.000000,0.496936,0.477053,0.507230,0.519077,...,0.509601,0.000000,0.000000,0.503846,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.766667,1.000000,0.333333,0.666667,0.754923,1.000000,0.758278,0.743632,0.755758,0.760335,...,0.754954,0.000000,0.000000,0.758137,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# 가설 설정 

## 1. 증상이 알츠하이머의 유/무에 영향을 미친다. 
- 방법 : 증상으로 알츠하이머의 유/무를 예측하는 로지스틱 회귀 모형을 만들어 각 독립변수들의 p-value를 통해 종속 변수에 영향을 미치는지 확인
- 결과 : 로지스틱 회귀 모형의 정확성이 0.64로 높은 성능을 보이지 않고, 각 변수들 역시 종속 변수에 영향을 미치지 않음을 p-value를 통해 확인

In [113]:
from statsmodels.formula.api import logit 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
df_symptom = df[['Confusion','Disorientation','PersonalityChanges','DifficultyCompletingTasks','Forgetfulness','Diagnosis']]
train,test = train_test_split(df_symptom,test_size = 0.3, random_state = 23)
model = logit('Diagnosis ~ Confusion + Disorientation + PersonalityChanges + DifficultyCompletingTasks + Forgetfulness',train).fit()
pred = model.predict(test)
pred_class = (pred > 0.5).astype(int)
print(classification_report(test['Diagnosis'],pred_class))

Optimization terminated successfully.
         Current function value: 0.648144
         Iterations 5
              precision    recall  f1-score   support

         0.0       0.64      1.00      0.78       416
         1.0       0.00      0.00      0.00       229

    accuracy                           0.64       645
   macro avg       0.32      0.50      0.39       645
weighted avg       0.42      0.64      0.51       645



/usr/local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### 불안

In [114]:

if model.pvalues['Confusion'] >= 0.05:
    print('p-value : ',model.pvalues['Confusion'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['Confusion'])
    print('p-value : ',model.pvalues['Confusion'])
    print('대립가설, 종속 변수에 영향을 미침')

p-value :  0.875904727700817
귀무가설, 종속 변수에 영향을 미치지 않음


### 방향 감각 상실

In [115]:
if model.pvalues['Disorientation'] >= 0.05:
    print('p-value : ',model.pvalues['Disorientation'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['Disorientation'])
    print('p-value : ',model.pvalues['Disorientation'])
    print('대립가설, 종속 변수에 영향을 미침')

p-value :  0.34566823922915324
귀무가설, 종속 변수에 영향을 미치지 않음


### 성격 변화

In [116]:
if model.pvalues['PersonalityChanges'] >= 0.05:
    print('p-value : ',model.pvalues['PersonalityChanges'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['PersonalityChanges'])
    print('p-value : ',model.pvalues['PersonalityChanges'])
    print('대립가설, 종속 변수에 영향을 미침')

p-value :  0.23377298755240006
귀무가설, 종속 변수에 영향을 미치지 않음


### 작업 완료 어려움

In [117]:
if model.pvalues['DifficultyCompletingTasks'] >= 0.05:
    print('p-value : ',model.pvalues['DifficultyCompletingTasks'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['DifficultyCompletingTasks'])
    print('p-value : ',model.pvalues['DifficultyCompletingTasks'])
    print('대립가설, 종속 변수에 영향을 미침')

p-value :  0.7507729249969415
귀무가설, 종속 변수에 영향을 미치지 않음


### 건망증

In [118]:
if model.pvalues['Forgetfulness'] >= 0.05:
    print('p-value : ',model.pvalues['Forgetfulness'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['Forgetfulness'])
    print('p-value : ',model.pvalues['Forgetfulness'])
    print('대립가설, 종속 변수에 영향을 미침')

p-value :  0.26296808097953495
귀무가설, 종속 변수에 영향을 미치지 않음


## 2. 인지 및 기능 평가 변수들이 알츠하이머의 유/무에 영향을 미친다.
- 방법 : 인지 및 기능 평가 변수들로 알츠하이머의 유/무를 예측하는 로지스틱 회귀 모형을 만들어 각 독립변수들의 p-value를 통해 종속 변수에 영향을 미치는지 확인
- 결과 1 : 로지스틱 회귀 모형의 정확성이 0.86로 높은 성능을 보이고, 각 변수들 역시 종속 변수에 영향을 미치다는 것을 p-value를 통해 확인
- 결과 2 : 간이 점수 상태 점수,기능 평가, 일상생활활동 점수는 종속변수에 긍정적인 영향을 미치는 것을 확인
- 결과 3 : 기억 장애 유무와 행동 문제 유무는 종속변수에 부정적인 영향을 미치는 것을 확인

In [119]:
from statsmodels.formula.api import logit 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
df_assessment = df[['MMSE','FunctionalAssessment','MemoryComplaints','BehavioralProblems','ADL','Diagnosis']]
train,test = train_test_split(df_assessment,test_size = 0.3, random_state = 23)
model = logit('Diagnosis ~ MMSE + FunctionalAssessment + MemoryComplaints + BehavioralProblems + ADL',train).fit()
pred = model.predict(test)
pred_class = (pred > 0.5).astype(int)
print(classification_report(test['Diagnosis'],pred_class))

Optimization terminated successfully.
         Current function value: 0.394660
         Iterations 7
              precision    recall  f1-score   support

         0.0       0.88      0.90      0.89       416
         1.0       0.81      0.78      0.80       229

    accuracy                           0.86       645
   macro avg       0.85      0.84      0.84       645
weighted avg       0.86      0.86      0.86       645



### 간이 점수 상태 점수

In [120]:
if model.pvalues['MMSE'] >= 0.05:
    print('p-value : ',model.pvalues['MMSE'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['MMSE'])
    print('p-value : ',model.pvalues['MMSE'])
    print('대립가설, 종속 변수에 영향을 미침')

회귀 계수 :  -3.2199395575701555
p-value :  1.015756590254141e-29
대립가설, 종속 변수에 영향을 미침


### 기능 평가

In [121]:

if model.pvalues['FunctionalAssessment'] >= 0.05:
    print('p-value : ',model.pvalues['FunctionalAssessment'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['FunctionalAssessment'])
    print('p-value : ',model.pvalues['FunctionalAssessment'])
    print('대립가설, 종속 변수에 영향을 미침')

회귀 계수 :  -4.041402753123401
p-value :  3.1231672277786595e-43
대립가설, 종속 변수에 영향을 미침


### 기억 장애 유무

In [122]:
if model.pvalues['MemoryComplaints'] >= 0.05:
    print('p-value : ',model.pvalues['MemoryComplaints'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['MemoryComplaints'])
    print('p-value : ',model.pvalues['MemoryComplaints'])
    print('대립가설, 종속 변수에 영향을 미침')

회귀 계수 :  2.36007705923585
p-value :  4.078091257854514e-36
대립가설, 종속 변수에 영향을 미침


### 행동 문제 유무

In [123]:
if model.pvalues['BehavioralProblems'] >= 0.05:
    print('p-value : ',model.pvalues['BehavioralProblems'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['BehavioralProblems'])
    print('p-value : ',model.pvalues['BehavioralProblems'])
    print('대립가설, 종속 변수에 영향을 미침')

회귀 계수 :  2.340244326684079
p-value :  6.562783796460691e-29
대립가설, 종속 변수에 영향을 미침


### 일상생활활동 점수

In [124]:

if model.pvalues['ADL'] >= 0.05:
    print('p-value : ',model.pvalues['ADL'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['ADL'])
    print('p-value : ',model.pvalues['ADL'])    
    print('대립가설, 종속 변수에 영향을 미침')

회귀 계수 :  -3.801097963889272
p-value :  4.561086954718619e-39
대립가설, 종속 변수에 영향을 미침


## 3. 임상 측정 결과 변수들이 알츠하이머의 유/무에 영향을 미친다.
- 방법 : 임상 측정 결과 변수들로 알츠하이머의 유/무를 예측하는 로지스틱 회귀 모형을 만들어 각 독립변수들의 p-value를 통해 종속 변수에 영향을 미치는지 확인
- 결과 1 : 로지스틱 회귀 모형의 정확성이 0.64로 높은 성능을 보이지 않고, 각 변수들 역시 HDL을 제외하고는 종속 변수에 영향을 미치지 않음을 p-value를 통해 확인
- 결과 2 : HDL의 경우, 종속 변수에 긍정적인 영향을 미침(회귀 계수 : 0.45)

In [125]:
from statsmodels.formula.api import logit 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
df_clinical = df[['SystolicBP','DiastolicBP','CholesterolTotal','CholesterolLDL','CholesterolHDL','CholesterolTriglycerides','BMI','Diagnosis']]
train,test = train_test_split(df_clinical,test_size = 0.3, random_state = 23)
model = logit('Diagnosis ~ SystolicBP + DiastolicBP + CholesterolTotal + CholesterolLDL + CholesterolHDL + CholesterolTriglycerides + BMI',train).fit()
pred = model.predict(test)
pred_class = (pred > 0.5).astype(int)
print(classification_report(test['Diagnosis'],pred_class))

Optimization terminated successfully.
         Current function value: 0.646139
         Iterations 5
              precision    recall  f1-score   support

         0.0       0.64      1.00      0.78       416
         1.0       0.00      0.00      0.00       229

    accuracy                           0.64       645
   macro avg       0.32      0.50      0.39       645
weighted avg       0.42      0.64      0.51       645



/usr/local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### 수축기 혈압

In [126]:
if model.pvalues['SystolicBP'] >= 0.05:
    print('p-value : ',model.pvalues['SystolicBP'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['SystolicBP'])
    print('p-value : ',model.pvalues['SystolicBP'])
    print('대립가설, 종속 변수에 영향을 미침')

p-value :  0.39752114802731064
귀무가설, 종속 변수에 영향을 미치지 않음


### 이완기 혈압

In [127]:
if model.pvalues['DiastolicBP'] >= 0.05:
    print('p-value : ',model.pvalues['DiastolicBP'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['DiastolicBP'])
    print('p-value : ',model.pvalues['DiastolicBP'])
    print('대립가설, 종속 변수에 영향을 미침')

p-value :  0.7486254145346884
귀무가설, 종속 변수에 영향을 미치지 않음


### 콜레스테롤 총량

In [128]:
if model.pvalues['CholesterolTotal'] >= 0.05:
    print('p-value : ',model.pvalues['CholesterolTotal'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['CholesterolTotal'])
    print('p-value : ',model.pvalues['CholesterolTotal'])
    print('대립가설, 종속 변수에 영향을 미침')

p-value :  0.7551283054649616
귀무가설, 종속 변수에 영향을 미치지 않음


### 콜레스테롤 LDL

In [129]:
if model.pvalues['CholesterolLDL'] >= 0.05:
    print('p-value : ',model.pvalues['CholesterolLDL'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['CholesterolLDL'])
    print('p-value : ',model.pvalues['CholesterolLDL'])
    print('대립가설, 종속 변수에 영향을 미침')

p-value :  0.23318477886695943
귀무가설, 종속 변수에 영향을 미치지 않음


### 콜레스테롤 HDL

In [130]:
if model.pvalues['CholesterolHDL'] >= 0.05:
    print('p-value : ',model.pvalues['CholesterolHDL'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['CholesterolHDL'])
    print('p-value : ',model.pvalues['CholesterolHDL'])
    print('대립가설, 종속 변수에 영향을 미침')

회귀 계수 :  0.45320468793932145
p-value :  0.01613822890661023
대립가설, 종속 변수에 영향을 미침


### 중성 지방

In [131]:
if model.pvalues['CholesterolTriglycerides'] >= 0.05:
    print('p-value : ',model.pvalues['CholesterolTriglycerides'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['CholesterolTriglycerides'])
    print('p-value : ',model.pvalues['CholesterolTriglycerides'])
    print('대립가설, 종속 변수에 영향을 미침')

p-value :  0.8666096626347534
귀무가설, 종속 변수에 영향을 미치지 않음


### BMI

In [132]:
if model.pvalues['BMI'] >= 0.05:
    print('p-value : ',model.pvalues['BMI'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['BMI'])
    print('p-value : ',model.pvalues['BMI'])
    print('대립가설, 종속 변수에 영향을 미침')

p-value :  0.34538593240432947
귀무가설, 종속 변수에 영향을 미치지 않음


## 4. 병력 변수들이 알츠하이머의 유/무에 영향을 미친다.
- 방법 : 임상 측정 결과 변수들로 알츠하이머의 유/무를 예측하는 로지스틱 회귀 모형을 만들어 각 독립변수들의 p-value를 통해 종속 변수에 영향을 미치는지 확인
- 결과 1 : 로지스틱 회귀 모형의 정확성이 0.64로 높은 성능을 보이지 않고, 각 변수들 역시 심혈관 질환과 당뇨병을 제외하고는 종속 변수에 영향을 미치지 않음을 p-value를 통해 확인
- 결과 2 : 심혈관질환의 경우 경우, 종속 변수에 긍정적인 영향을 미침(회귀 계수 : 0.30)
- 결과 3 : 당뇨병의 경우, 종속 변수에 부정적인 영향을 미침(회귀 계수 : -0.35)

In [151]:
from statsmodels.formula.api import logit 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
df_history = df[['FamilyHistoryAlzheimers','CardiovascularDisease','Diabetes','Depression','HeadInjury','Hypertension','Diagnosis']]
train,test = train_test_split(df_history,test_size = 0.3, random_state = 23)
model = logit('Diagnosis ~ FamilyHistoryAlzheimers + CardiovascularDisease + Diabetes + Depression + HeadInjury + Hypertension',train).fit()
pred = model.predict(test)
pred_class = (pred > 0.5).astype(int)
print(classification_report(test['Diagnosis'],pred_class))

Optimization terminated successfully.
         Current function value: 0.644727
         Iterations 5
              precision    recall  f1-score   support

         0.0       0.64      1.00      0.78       416
         1.0       0.00      0.00      0.00       229

    accuracy                           0.64       645
   macro avg       0.32      0.50      0.39       645
weighted avg       0.42      0.64      0.51       645



/usr/local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### 알츠하이머 가족력

In [134]:
if model.pvalues['FamilyHistoryAlzheimers'] >= 0.05:
    print('p-value : ',model.pvalues['FamilyHistoryAlzheimers'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['FamilyHistoryAlzheimers'])
    print('p-value : ',model.pvalues['FamilyHistoryAlzheimers'])
    print('대립가설, 종속 변수에 영향을 미침')

p-value :  0.1355331419321811
귀무가설, 종속 변수에 영향을 미치지 않음


### 심혈관 질환

In [152]:
if model.pvalues['CardiovascularDisease'] >= 0.05:
    print('p-value : ',model.pvalues['CardiovascularDisease'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['CardiovascularDisease'])
    print('p-value : ',model.pvalues['CardiovascularDisease'])
    print('대립가설, 종속 변수에 영향을 미침')

회귀 계수 :  0.29655102412901785
p-value :  0.047120228004136004
대립가설, 종속 변수에 영향을 미침


### 당뇨병

In [153]:
if model.pvalues['Diabetes'] >= 0.05:
    print('p-value : ',model.pvalues['Diabetes'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['Diabetes'])
    print('p-value : ',model.pvalues['Diabetes'])
    print('대립가설, 종속 변수에 영향을 미침')

회귀 계수 :  -0.34606135111479297
p-value :  0.03164161784077769
대립가설, 종속 변수에 영향을 미침


### 우울증

In [137]:
if model.pvalues['Depression'] >= 0.05:
    print('p-value : ',model.pvalues['Depression'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['Depression'])
    print('p-value : ',model.pvalues['Depression'])
    print('대립가설, 종속 변수에 영향을 미침')

p-value :  0.5541258134913704
귀무가설, 종속 변수에 영향을 미치지 않음


### 두부 손상

In [138]:
if model.pvalues['HeadInjury'] >= 0.05:
    print('p-value : ',model.pvalues['HeadInjury'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['HeadInjury'])
    print('p-value : ',model.pvalues['HeadInjury'])
    print('대립가설, 종속 변수에 영향을 미침')

p-value :  0.18204318710895218
귀무가설, 종속 변수에 영향을 미치지 않음


### 고혈압

In [139]:
if model.pvalues['Hypertension'] >= 0.05:
    print('p-value : ',model.pvalues['Hypertension'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['Hypertension'])
    print('p-value : ',model.pvalues['Hypertension'])
    print('대립가설, 종속 변수에 영향을 미침')

p-value :  0.29027525901044904
귀무가설, 종속 변수에 영향을 미치지 않음


## 5. 일상생활 변수들이 알츠하이머의 유/무에 영향을 미친다.
- 방법 : 일상생활 변수들로 알츠하이머의 유/무를 예측하는 로지스틱 회귀 모형을 만들어 각 독립변수들의 p-value를 통해 종속 변수에 영향을 미치는지 확인
- 결과 1 : 로지스틱 회귀 모형의 정확성이 0.64로 높은 성능을 보이지 않고, 각 변수들 역시 수면의 질을 제외하고는 종속 변수에 영향을 미치지 않음을 p-value를 통해 확인
- 결과 2 : 수면의 질의 경우, 종속 변수여 부정적인 영향을 미침(회귀 계수 : -0.38)

In [140]:
from statsmodels.formula.api import logit 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
df_lifestyle = df[['Smoking','AlcoholConsumption','PhysicalActivity','DietQuality','SleepQuality','Diagnosis']]
train,test = train_test_split(df_lifestyle,test_size = 0.3, random_state = 23)
model = logit('Diagnosis ~ Smoking + AlcoholConsumption + PhysicalActivity + DietQuality + SleepQuality',train).fit()
pred = model.predict(test)
pred_class = (pred > 0.5).astype(int)
print(classification_report(test['Diagnosis'],pred_class))

Optimization terminated successfully.
         Current function value: 0.647616
         Iterations 4
              precision    recall  f1-score   support

         0.0       0.64      1.00      0.78       416
         1.0       0.00      0.00      0.00       229

    accuracy                           0.64       645
   macro avg       0.32      0.50      0.39       645
weighted avg       0.42      0.64      0.51       645



/usr/local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### 흡연 유/무

In [141]:
if model.pvalues['Smoking'] >= 0.05:
    print('p-value : ',model.pvalues['Smoking'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['Smoking'])
    print('p-value : ',model.pvalues['Smoking'])
    print('대립가설, 종속 변수에 영향을 미침')

p-value :  0.6393586879619717
귀무가설, 종속 변수에 영향을 미치지 않음


### 알코올 소비량

In [142]:
if model.pvalues['AlcoholConsumption'] >= 0.05:
    print('p-value : ',model.pvalues['AlcoholConsumption'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['AlcoholConsumption'])
    print('p-value : ',model.pvalues['AlcoholConsumption'])
    print('대립가설, 종속 변수에 영향을 미침')

p-value :  0.8262207964164993
귀무가설, 종속 변수에 영향을 미치지 않음


### 신체 활동

In [143]:
if model.pvalues['PhysicalActivity'] >= 0.05:
    print('p-value : ',model.pvalues['PhysicalActivity'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['PhysicalActivity'])
    print('p-value : ',model.pvalues['PhysicalActivity'])
    print('대립가설, 종속 변수에 영향을 미침')

p-value :  0.47831367338710895
귀무가설, 종속 변수에 영향을 미치지 않음


### 식단의 질

In [144]:
if model.pvalues['DietQuality'] >= 0.05:
    print('p-value : ',model.pvalues['DietQuality'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['DietQuality'])
    print('p-value : ',model.pvalues['DietQuality'])
    print('대립가설, 종속 변수에 영향을 미침')

p-value :  0.6681260593766596
귀무가설, 종속 변수에 영향을 미치지 않음


### 수면의 질

In [145]:
if model.pvalues['SleepQuality'] >= 0.05:
    print('p-value : ',model.pvalues['SleepQuality'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['SleepQuality'])
    print('p-value : ',model.pvalues['SleepQuality'])
    print('대립가설, 종속 변수에 영향을 미침')

회귀 계수 :  -0.38408533048764065
p-value :  0.039334513479360636
대립가설, 종속 변수에 영향을 미침


## 6. 인적정보 변수들이 알츠하이머의 유/무에 영향을 미친다.
- 방법 : 인적정보 변수들로 알츠하이머의 유/무를 예측하는 로지스틱 회귀 모형을 만들어 각 독립변수들의 p-value를 통해 종속 변수에 영향을 미치는지 확인
- 결과 : 로지스틱 회귀 모형의 정확성이 0.64로 높은 성능을 보이지 않고, 각 변수들 역시 종속 변수에 영향을 미치지 않음을 p-value를 통해 확인


In [146]:
from statsmodels.formula.api import logit 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
df_lifestyle = df[['Age','Gender','Ethnicity','EducationLevel','Diagnosis']]
train,test = train_test_split(df_lifestyle,test_size = 0.3, random_state = 23)
model = logit('Diagnosis ~ Age + Gender + Ethnicity + EducationLevel',train).fit()
pred = model.predict(test)
pred_class = (pred > 0.5).astype(int)
print(accuracy_score(test['Diagnosis'],pred_class))
print(classification_report(test['Diagnosis'],pred_class))


Optimization terminated successfully.
         Current function value: 0.648526
         Iterations 4
0.6449612403100775
              precision    recall  f1-score   support

         0.0       0.64      1.00      0.78       416
         1.0       0.00      0.00      0.00       229

    accuracy                           0.64       645
   macro avg       0.32      0.50      0.39       645
weighted avg       0.42      0.64      0.51       645



/usr/local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### 나이

In [147]:
if model.pvalues['Age'] >= 0.05:
    print('p-value : ',model.pvalues['Age'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['Age'])
    print('p-value : ',model.pvalues['Age'])
    print('대립가설, 종속 변수에 영향을 미침')

p-value :  0.9941044695660038
귀무가설, 종속 변수에 영향을 미치지 않음


### 성별

In [148]:
if model.pvalues['Gender'] >= 0.05:
    print('p-value : ',model.pvalues['Gender'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['Gender'])
    print('p-value : ',model.pvalues['Gender'])
    print('대립가설, 종속 변수에 영향을 미침')

p-value :  0.28041903262942536
귀무가설, 종속 변수에 영향을 미치지 않음


### 인종

In [149]:
if model.pvalues['Ethnicity'] >= 0.05:
    print('p-value : ',model.pvalues['Ethnicity'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['Ethnicity'])
    print('p-value : ',model.pvalues['Ethnicity'])
    print('대립가설, 종속 변수에 영향을 미침')

p-value :  0.3986516686259841
귀무가설, 종속 변수에 영향을 미치지 않음


### 교육 수준

In [150]:
if model.pvalues['EducationLevel'] >= 0.05:
    print('p-value : ',model.pvalues['EducationLevel'])
    print('귀무가설, 종속 변수에 영향을 미치지 않음')
else:
    print('회귀 계수 : ',model.params['EducationLevel'])
    print('p-value : ',model.pvalues['EducationLevel'])
    print('대립가설, 종속 변수에 영향을 미침')

p-value :  0.48610066829304943
귀무가설, 종속 변수에 영향을 미치지 않음


# 결론 
- ## 각 독립 변수 그룹들을 사용하여 만든 로지스티 회귀 모델의 성능 지표를 확인한 결과, 인지 및 기능 평가 변수 그룹을 제외한 변수 그룹들을 사용한 모델들은 모든 데이터를 0으로 분류함
- ## 데이터 불균형으로 인해 이러한 결과가 나온 것으로 판단이 되고, 이를 개선하고 다시 가설 검정을 실행